Packages:

In [43]:
import glob
import os
import warnings
 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from pydicom.data import get_testdata_files
from sklearn.model_selection import train_test_split

from IPython.display import Markdown, display

 # Import functions from the module
import importlib
import help_files._0_definitions 
import  help_files._1_visuals_script
# import  help_files._01_load_data
 # Reload the module to apply the changes to the script
importlib.reload(help_files._0_definitions)
importlib.reload(help_files._1_visuals_script)
# importlib.reload(help_files._01_load_data)
import  help_files._1_visuals_script  as pauls_vs
# Group by 'condition', 'level', and 'severity' and count occurrences
from help_files._0_definitions import count_severity_by_condition_level 
# Define the path
from pathlib import Path

pd.set_option("display.width", 1000)  # Set a large width to prevent line wrapping
 

In [44]:
### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_definitions.py") as file:
    exec(file.read())

### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_run_definitions.py") as file:
    exec(file.read())

In [45]:
# Specify the file names and DataFrame variable names
file_names = ["test_df_2.csv"]
# Load the data from the CSV files
dataframes = [pd.read_csv(data_path_vor / file_name) for file_name in file_names]
# Load train_df as a DataFrame
test_df = dataframes[0]  # dataframes[0] accesses the first (and in this case, only) DataFrame in that list.
 

In [46]:
if whole_data_set == True:
    print("Using the whole data set")
else:
    test_df = test_df.sample(n=50, random_state=RSEED)
    display(Markdown('<span style="color:red"> this is a experimentation sample : 48692</span>'))

Using the whole data set


Restore parameters and metrtics from mlflow

I should take prdicted probabilities and not predicted classes: output scores (logits) are converted into probabilities using the softmax function

In [47]:
import mlflow
import mlflow.pytorch
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.models as models
from torchvision import transforms
import pydicom
import cv2
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

# Set the experiment name
 
loaded_model = mlflow.pytorch.load_model("runs:/aa4fb3c2e81a4697997b99cb0b39eb48/model")
model_uri = f"runs:/{run_id}/model"

# Load the model
model = mlflow.pytorch.load_model(model_uri)
model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))  # Move model to appropriate device
model.eval()  # Set the model to evaluation mode

# Prepare data
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class MRIDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        self.data['severity'] = self.data['severity'].astype(int)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        image_path = row['image_path']
        label = row['severity']
        dicom_image = pydicom.dcmread(image_path)
        image = dicom_image.pixel_array.astype(float)
        image = (image / image.max() * 255).astype('uint8')
        if len(image.shape) == 2:  # Grayscale
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        image_tensor = self.transform(image) if self.transform else torch.from_numpy(image).permute(2, 0, 1)
        return image_tensor, torch.tensor(label).long()

    def __len__(self):
        return len(self.data)

predicting probabilities for test data

In [48]:
test_data = test_df

In [49]:
test_dataset = MRIDataset(data=test_data, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Inference loop with probability extraction
results = []
probabilities_list = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)  # Ensure labels are also on the correct device
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)  # Calculate class probabilities
        _, predicted_classes = torch.max(outputs, 1)
        
        # Append predictions and probabilities
        results.append(predicted_classes.item())
        probabilities_list.append(probabilities.cpu().numpy())



Using device: cpu


In [50]:
# Print predicted classes and their probabilities
for i, (pred, probs) in enumerate(zip(results, probabilities_list)):
    print(f"Test image {i}: Predicted class {pred}, Probabilities: {probs}")

# Additional code to plot the confusion matrix can stay as-is:
true_labels = []
predicted_labels = []

for images, labels in test_loader:
    images = images.to(device)
    true_labels.extend(labels.numpy())

    with torch.no_grad():
        outputs = model(images)
        _, predicted_classes = torch.max(outputs, 1)
        predicted_labels.extend(predicted_classes.cpu().numpy())

true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

print("True Labels Unique Values:", np.unique(true_labels))
print("Predicted Labels Unique Values:", np.unique(predicted_labels))


Test image 0: Predicted class 0, Probabilities: [[0.88555676 0.11444326]]
Test image 1: Predicted class 0, Probabilities: [[0.728574   0.27142602]]
Test image 2: Predicted class 0, Probabilities: [[0.5479448 0.4520552]]
Test image 3: Predicted class 0, Probabilities: [[0.64558    0.35442004]]
Test image 4: Predicted class 1, Probabilities: [[0.26947173 0.7305283 ]]
Test image 5: Predicted class 0, Probabilities: [[0.91364133 0.08635864]]
Test image 6: Predicted class 0, Probabilities: [[0.94004554 0.05995454]]
Test image 7: Predicted class 0, Probabilities: [[0.9128294  0.08717064]]
Test image 8: Predicted class 0, Probabilities: [[0.87387526 0.12612472]]
Test image 9: Predicted class 0, Probabilities: [[0.91813946 0.08186051]]
Test image 10: Predicted class 0, Probabilities: [[0.601062   0.39893803]]
Test image 11: Predicted class 0, Probabilities: [[0.8658457 0.1341543]]
Test image 12: Predicted class 0, Probabilities: [[0.9122853  0.08771463]]
Test image 13: Predicted class 0, Proba

In [52]:
# Convert the list of probabilities to a numpy array for easier manipulation
probabilities_array = np.vstack(probabilities_list)

# Ensure the length of probabilities_array matches the length of test_df
probabilities_array = probabilities_array[:len(test_df)]

# Add the probabilities to the test_df DataFrame
test_df['Probability_Class_0'] = probabilities_array[:, 0]
test_df['Probability_Class_1'] = probabilities_array[:, 1]

# Round the probabilities to 4 decimal places
test_df['Probability_Class_0'] = test_df['Probability_Class_0'].round(4)
test_df['Probability_Class_1'] = test_df['Probability_Class_1'].round(4)

# Add the predicted classes to the test_df DataFrame
test_df['Predicted_Class'] = results



# Display the updated DataFrame
 
test_df.head()

,study_id,severity,condition,level,series_id,x,y,image_path,missing_image,Probability_Class_0,Probability_Class_1,Predicted_Class
0,4003253,0,0,l5/s1,1.054714e+09,197.100569,289.457306,data/train_images_origin/4003253/1054713880/3.dcm,False,0.8856,0.1144,0
1,4003253,0,0,l5/s1,1.054714e+09,197.100569,289.457306,data/train_images_origin/4003253/702807833/3.dcm,False,0.7286,0.2714,0
2,4003253,0,0,l5/s1,1.054714e+09,197.100569,289.457306,data/train_images_origin/4003253/1054713880/14...,False,0.5479,0.4521,0
3,4646740,1,0,l5/s1,3.486248e+09,235.980844,360.313610,data/train_images_origin/4646740/3486248476/2.dcm,False,0.6456,0.3544,0
4,4646740,1,0,l5/s1,3.486248e+09,235.980844,360.313610,data/train_images_origin/4646740/3486248476/1.dcm,False,0.2695,0.7305,1


In [54]:
# Group by severity and sum the Probability_Class_1
severity_prob_sum_all_classes = test_df.groupby('severity')[['Probability_Class_0', 'Probability_Class_1']].mean()

# Print the result
severity_prob_sum_all_classes
# Use severity_prob_sum_all_classes as confusion matrix
conf_matrix_prob_df = severity_prob_sum_all_classes 
# Create a confusion matrix from severity and predicted class
conf_matrix_severity_pred = confusion_matrix(test_df['severity'], test_df['Predicted_Class'])

# Create a DataFrame from the confusion matrix
conf_matrix_severity_pred_df = pd.DataFrame(
    conf_matrix_severity_pred, 
    index=[f"Actual {i}" for i in range(len(conf_matrix_severity_pred))], 
    columns=[f"Predicted {i}" for i in range(len(conf_matrix_severity_pred[0]))]
)

# Print the confusion matrix DataFrame
print(conf_matrix_severity_pred_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_severity_pred_df.style.background_gradient(cmap='Blues')
# Print the confusion matrix DataFrame
print(conf_matrix_prob_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_prob_df.style.background_gradient(cmap='Blues')


          Predicted 0  Predicted 1
Actual 0         3536         2095
Actual 1         3049         2489
          Probability_Class_0  Probability_Class_1
severity                                          
0                    0.595345             0.404655
1                    0.532448             0.467552


,Probability_Class_0,Probability_Class_1
severity,,
0,0.595345,0.404655
1,0.532448,0.467552


In [ ]:
conf_matrix_severity_pred_df.style.background_gradient(cmap='Blues')

In [55]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np

# Ensure true_labels and predicted_labels have the same length
min_length = min(len(true_labels), len(predicted_labels))
true_labels = true_labels[:min_length]
predicted_labels = predicted_labels[:min_length]

# Generate the confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Create a DataFrame from the confusion matrix
conf_matrix_df = pd.DataFrame(
    conf_matrix, 
    index=[f"Actual {i}" for i in range(len(conf_matrix))], 
    columns=[f"Predicted {i}" for i in range(len(conf_matrix[0]))]
)

# Print the confusion matrix DataFrame
print(conf_matrix_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_df.style.background_gradient(cmap='Blues')


          Predicted 0  Predicted 1
Actual 0         3549         2082
Actual 1         3064         2474


,Predicted 0,Predicted 1
Actual 0,3549,2082
Actual 1,3064,2474


In [56]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
# Calculate precision, recall, and accuracy using mean_probabilities_df
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")


Precision: 0.5398
Recall: 0.5393
Accuracy: 0.5393
